Oseen Equation
===

Import Netgen/NGSolve Python modules:

In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from math import log
import numpy as np
import scipy.sparse as sp
SetHeapSize(int(1e9))   

In [2]:
1/16

0.0625

In [3]:
N = 1
T = 0.2
tau = T/N 
gamma0 = 1
gamma1 = 1
gamma_dual = 1e-5
gamma_M = 1e5
# maxh = 0.0375 + (0.075-0.0375)/2
#print(maxh)
maxh = 0.6
solver = "pardiso"
#solver = "umfpack"

In [4]:
#maxh

The unit_square is a predefined domain, use Netgen to generate a mesh:

In [5]:
def freedofs_converter(fd):
    frees = [] 
    for i in range(len(fd)):
        if fd[i]:
            frees.append(i)
    return frees

def cond_est(a,frees):
    rows,cols,vals = a.mat.COO()
    A = sp.csr_matrix((vals,(rows,cols)))
    A_red = A.todense()[frees,:][:,frees] 
    return np.linalg.cond(A_red)

In [6]:
def GetMeshDataAllAround(maxh):
    geo = SplineGeometry()
    # data domain
    p1 = geo.AppendPoint (0,0)
    p2 = geo.AppendPoint (1,0)
    p4 = geo.AppendPoint (0.75,0.75)
    p5 = geo.AppendPoint (0.75,0.25)
    p6 = geo.AppendPoint (0.25,0.25)
    p7 = geo.AppendPoint (0.25,0.75)
    p11 = geo.AppendPoint(1.0,1.0)
    p12 = geo.AppendPoint(0.0,1.0)
    # omega
    geo.Append (["line", p1, p2], leftdomain=1, rightdomain=0,bc="bc_Omega")
    #l1 = geo.Append (["line", p1, p2], leftdomain=1, rightdomain=0,bc="perUp")
    geo.Append (["line", p2, p11], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p11, p12], leftdomain=1, rightdomain=0,bc="bc_Omega")
    #geo.Append (["line", p12, p1], leftdomain=1, rightdomain=0,bc="perDown", copy=l1)
    geo.Append (["line", p12, p1], leftdomain=1, rightdomain=0,bc="bc_Omega")
    # only_B
    geo.Append (["line", p6, p5], leftdomain=2, rightdomain=1)
    geo.Append (["line", p5, p4], leftdomain=2, rightdomain=1)
    geo.Append (["line", p4, p7], leftdomain=2, rightdomain=1)
    geo.Append (["line", p7, p6], leftdomain=2, rightdomain=1)
    geo.SetMaterial(1, "omega")
    geo.SetMaterial(2, "only_B")
    return geo.GenerateMesh(maxh=maxh)

In [7]:
def GetMeshDataLeft(maxh):
    geo = SplineGeometry()
    p1 = geo.AppendPoint (0,0)
    p2 = geo.AppendPoint (0.25,0)
    p3 = geo.AppendPoint (0.25,1)
    p4 = geo.AppendPoint (0,1)
    p5 = geo.AppendPoint (1,0)
    p6 = geo.AppendPoint (1,1)
    # omega 
    geo.Append (["line", p1, p2], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p2, p3], leftdomain=1, rightdomain=2)
    geo.Append (["line", p3, p4], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p4, p1], leftdomain=1, rightdomain=0,bc="bc_Omega")
    # only_B 
    geo.Append (["line", p2, p5], leftdomain=2, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p5, p6], leftdomain=2, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p6, p3], leftdomain=2, rightdomain=0,bc="bc_Omega")
    geo.SetMaterial(1, "omega")
    geo.SetMaterial(2, "only_B")
    return geo.GenerateMesh(maxh=maxh)

In [8]:
#mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh = Mesh(GetMeshDataAllAround(maxh) )
# mesh = Mesh(GetMeshDataLeft(maxh))
h = specialcf.mesh_size
n = specialcf.normal(2)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [9]:
t = Parameter(0.0)

u_sol = CoefficientFunction( (  sin(2*pi*x)*cos(2*pi*y)*exp(-8*pi**2*t), 
                              - cos(2*pi*x)*sin(2*pi*y)*exp(-8*pi**2*t) ) )
p_sol = 0.25*(cos(4*pi*x)+cos(4*pi*y))*exp(-16*pi**2*t)
#beta = CoefficientFunction( ( 1.0 , 1.0 ) )

# u_sol = CoefficientFunction( ( cos(t)*20*x*y**3, 
#                                cos(t)*(5*x**4-5*y**4 )   ) )
u_exact = u_sol 
# p_sol = (60*x**2*y-20*y**3-5)*cos(t)
beta_ref = u_sol 

nu = 1 
u_exact = u_sol
p_exact = p_sol
rhs = CoefficientFunction(( u_exact[0].Diff(t) + (u_exact[0]*u_exact[0].Diff(x) + u_exact[1]*u_exact[0].Diff(y)) 
                           - nu* ( u_exact[0].Diff(x).Diff(x) + u_exact[0].Diff(y).Diff(y) ) +  p_exact.Diff(x) ,
                           u_exact[1].Diff(t) + (u_exact[0]*u_exact[1].Diff(x) + u_exact[1]*u_exact[1].Diff(y)) 
                           - nu* ( u_exact[1].Diff(x).Diff(x) + u_exact[1].Diff(y).Diff(y) ) +  p_exact.Diff(y)
                          ) )
#print(rhs)

# u_sol = CoefficientFunction( (   2*cos(t)*sin(pi*x)*sin(pi*x)*y*(1-y)*(1-2*y), 
#                             (-pi)*(cos(t))*sin(2*pi*x)*(y**4-2*y**3+y**2) ) )
# p_sol = sin(pi*x)*cos(pi*y)*cos(t) 
# beta = CoefficientFunction( ( 1.0 , 1.0 ) )
# rhs = CoefficientFunction( ( -2*sin(t)*sin(pi*x)*sin(pi*x)*y*(1-y)*(1-2*y)
#        -4*pi**2*cos(2*pi*x)*cos(t)*(2*y**3-3*y**2+y)-2*sin(pi*x)**2*cos(t)*(12*y-6)+pi*cos(pi*x)*cos(pi*y)*cos(t)  , 
#                            pi*sin(2*pi*x)*(y**4-2*y**3+y**2)*sin(t)
#       -(4*pi**3*sin(2*pi*x)*(y**4-2*y**3+y**2)*cos(t)-pi*sin(2*pi*x)*(12*y**2-12*y+2)*cos(t))
#       -pi*sin(pi*x)*sin(pi*y)*cos(t) )  )



In [10]:
div_sol = u_exact[0].Diff(x) + u_exact[1].Diff(y) 
#div_sol(mesh(0.4,0.4))
div_square_L2 = Integrate(div_sol**2,mesh,order=12)
print("div_square_L2 = ", div_square_L2)

div_square_L2 =  1.179067201279737e-31


In [11]:
#Draw(rhs,mesh )
Draw(u_sol[0] ,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Define a finite element space on that mesh. 

In [12]:
#fes = H1(mesh, order=3, dirichlet="left|right|bottom|top")
fes_NC = FESpace("nonconforming",mesh, dirichlet="bc_Omega", dgjumps = True) 
fes_lam = NumberSpace(mesh)
fes_L2 = L2(mesh, order=0)
fes_primal_vel = FESpace([fes_NC*fes_NC for i in range(N+1) ])
fes_primal_pressure = FESpace([ fes_L2 for i in range(N) ])
fes_dual_vel = FESpace([fes_NC*fes_NC for i in range(N) ])
fes_dual_pressure = FESpace([ fes_L2 for i in range(N) ])
fes_primal_lam = FESpace([fes_lam for i in range(N)])
fes_dual_lam = FESpace([fes_lam for i in range(N)])
X = FESpace( [fes_primal_vel, fes_primal_pressure,fes_primal_lam, fes_dual_vel, fes_dual_pressure,fes_dual_lam])
print ("X-ndof = {0}".format(X.ndof ))

X-ndof = 180


In [13]:
#gfu = GridFunction(X)
#Draw( gfu.components[0].components[N].components[1], mesh)
#print(gfu.components[0].dim)
#help(gfu)
#help(gfu.components[1])
#print(gfu.components[0][0].FV().NumPy())
#print(gfu.components[0][0].dim)
#help( gfu.components[0][0])
#print(len(gfu.components[0][0] ))
#Draw(gfu.components[1],mesh )

In [14]:
#help(gfu.components[0].components[0].components[0])

In [15]:
len(X.TrialFunction()) 

6

In [16]:
u, pp, llam, zz, yyy, xxi =  X.TrialFunction()
v, qq, mmu, ww, xxx, eeta =  X.TestFunction()
#print(len(u[0]) )
#print

p = [None] + [pp[i] for i in range(len(pp)) ]
z = [None] + [zz[i] for i in range(len(zz)) ]
yy = [None] + [yyy[i] for i in range(len(yyy)) ]
lam = [None] + [llam[i] for i in range(len(llam)) ]
xi = [None] + [xxi[i] for i in range(len(xxi)) ]

q = [None] + [qq[i] for i in range(len(qq)) ]
w = [None] + [ww[i] for i in range(len(ww)) ]
xx = [None] + [xxx[i] for i in range(len(xxx)) ]
mu = [None] + [mmu[i] for i in range(len(mmu)) ]
eta = [None] + [eeta[i] for i in range(len(eeta)) ]

#help(xx)

In [17]:
def IP(u,v,nabla=False):
    if nabla:
        return sum( [ grad(u[i])*grad(v[i]) for i in range(len(u))] )
    else:
        return sum( [u[i]*v[i] for i in range(len(u))] )

def IP_conv(u,v,beta):
    return beta[0]*grad(u[0])[0]*v[0]+beta[1]*grad(u[0])[1]*v[0]+beta[0]*grad(u[1])[0]*v[1]+beta[1]*grad(u[1])[1]*v[1]
    
def IP_ut_v(u_cur,u_prev,v):
    return sum( [ (u_cur[i] - u_prev[i] ) * v[i] for i in range(len(u_cur))] )
    #help(IP(u[0],v[0]))
def IP_mixed_stab(u_cur,u_prev,v_cur,v_prev):
    return sum( [ ( grad(u_cur[i]) - grad(u_prev[i]) ) * ( grad(v_cur[i]) - grad(v_prev[i]) )  for i in range(len(u_cur))] )

def IP_CIP(u,v):
    return sum( [ (u[i] - u[i].Other()) * (v[i] - v[i].Other()) for i in range(len(u))  ] )

def IP_jump_avg(u,v):
    return sum( [ (u[i] - u[i].Other()) * 0.5 * (v[i] + v[i].Other()) for i in range(len(u))  ] )

def IP_divu_q(u,q):
    u1_dx = grad(u[0])[0]
    u2_dy = grad(u[1])[1]
    div_u = u1_dx + u2_dy
    return div_u * q  

In [18]:
gfu = GridFunction(X)
beta_prev = GridFunction(X)
#beta_prev.vec.FV().NumPy()[:] = 1e1*np.random.rand(len( beta_prev.vec.FV().NumPy() ))[:]
#beta_prev.Set()
beta_prev.vec.FV().NumPy()[:] = 0.0
#for i in range(0,N+1):
#    t.Set(tau*i)
#    beta_prev.components[0].components[i].components[0].Set( u_sol[0]  )
#    beta_prev.components[0].components[i].components[1].Set( u_sol[1]  )

In [19]:
def SolveOseen(beta_prev):
    
    a = BilinearForm(X,symmetric=False)

    # add mean value pressure constraint 
    for i in range(1,N+1):
        a += (mu[i] * p[i] + lam[i] * q[i]) * dx  
        a += (eta[i] * yy[i] + xi[i] * xx[i]) * dx 

    # A1 
    for i in range(1,N+1):
        beta = CoefficientFunction((beta_prev.components[0].components[i-1].components[0], beta_prev.components[0].components[i-1].components[1] ) ) 
        
        a += IP_ut_v(u[i],u[i-1],w[i]) * dx
        a += tau * IP(u[i],w[i],nabla=True) * dx 
        a += tau * IP_conv(u[i], w[i],beta) * dx 
        a += tau *(-1)*IP_divu_q(w[i],p[i]) * dx
        a += tau * IP_divu_q(u[i],xx[i]) * dx 
        a +=  (-1)*tau * InnerProduct(beta , n ) * IP_jump_avg(u[i],w[i]) * dx(skeleton=True)
        #a +=  tau * (0.5*IfPos(InnerProduct(beta,n),InnerProduct(beta,n),-InnerProduct(beta,n))) * IP_CIP(u[i],w[i]) * dx(skeleton=True)

    # A2 
    a += gamma0 * h**2 * IP(u[0],v[0],nabla=True) * dx
    for i in range(1,N+1):
        beta = CoefficientFunction((beta_prev.components[0].components[i-1].components[0], beta_prev.components[0].components[i-1].components[1] ) ) 
    #     a += gamma_M * tau * IP(u[i],v[i]) * dx(definedon=mesh.Materials("omega"))
        a += gamma_M * tau * IP(u[i],v[i]) * dx(definedon=mesh.Materials("only_B"))
        a += gamma1 * tau * IP_mixed_stab(u[i],u[i-1],v[i],v[i-1]) * dx 
        a +=  tau * (1/h)  * IP_CIP(u[i],v[i]) * dx(skeleton=True)
        a +=  (-1)*tau * InnerProduct(beta , n ) * IP_jump_avg(v[i],z[i]) * dx(skeleton=True)
        #a +=  tau * (0.5*IfPos(InnerProduct(beta,n),InnerProduct(beta,n),-InnerProduct(beta,n))) * IP_CIP(v[i],z[i]) * dx(skeleton=True)
        a += IP_ut_v(v[i],v[i-1],z[i]) * dx
        a += tau * IP(v[i],z[i],nabla=True) * dx 
        a += tau * IP_conv(v[i], z[i],beta) * dx 

        a += (-1)*IP_divu_q(z[i],q[i]) * dx 
        a += IP_divu_q(v[i],yy[i]) * dx
        
    f = LinearForm(X)
    for i in range(1,N+1):
        t.Set(tau*i)
        f += tau * IP(w[i],rhs,nabla=False)  * dx
        f +=  gamma_M * tau * IP(v[i],u_sol) * dx(definedon=mesh.Materials("only_B"))

    with TaskManager():
        a.Assemble()
        f.Assemble()
    
    gfu.vec.FV().NumPy()[:] = 0.0
    for i in range(0,N+1):
        t.Set(tau*i)
        gfu.components[0].components[i].components[0].Set( u_sol[0] , BND )
        gfu.components[0].components[i].components[1].Set( u_sol[1] , BND )
        #Draw( gfu.components[0].components[i].components[0] , mesh )
    
    res = f.vec - a.mat * gfu.vec
    gfu.vec.data += a.mat.Inverse(X.FreeDofs(),inverse=solver) * res 
    
    uhx = gfu.components[0].components[N].components[0]
    uhy = gfu.components[0].components[N].components[1]
    uh = [uhx,uhy]
    p_primal = gfu.components[1].components[N-1]
    p_dual = gfu.components[4].components[N-1]
    
    t.Set(tau*N)
    Draw (u_sol[0] , mesh);
    
    Draw (uhx, mesh);
    #input("")
    
    # compute error 
    t.Set(tau*N)
    error=sqrt(Integrate( (u_sol[0]-uhx)**2 + (u_sol[1]-uhy)**2 , mesh))
    ref_error=sqrt(Integrate( (u_sol[0])**2 + (u_sol[1])**2 , mesh))
#     print ("L2-error:", error/ref_error)
    print ("L2-error:", error)
    beta_prev.vec.data = gfu.vec.data 
#     return error/ref_error
    return error
    

In [20]:
l2_errors = [ ]
# Fixed point iteration 
maxiter = 5 
for i in range(maxiter): 
    l2err = SolveOseen(beta_prev)
    l2_errors.append(l2err) 

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

L2-error: 0.012773631926557416


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

L2-error: 0.01278431387959717


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

L2-error: 0.012784313881903769


In [21]:
l2_errors

[0.012773631926557416,
 0.012783603937571048,
 0.012784313605902166,
 0.01278431387959717,
 0.012784313881903769]

In [22]:
print([ abs(l2_errors[i] - l2_errors[-1] ) for i in range(len(l2_errors)-1 ) ])

[1.0681955346352606e-05, 7.099443327211741e-07, 2.760016035163737e-10, 2.3065993559612252e-12]


In [23]:
# Oseen 
l2_errors = [0.012784313881903769, 0.0005093642204630777,3.167830015733365e-05, 1.9388585623755054e-06]
#hm = [ 0.8,0.4,0.2,0.1,0.05]
#Ns = [1,2,4,8,16]
eoc = [ log(l2_errors[i-1]/l2_errors[i])/log(2) for i in range(1,len(l2_errors))]
print(eoc)

[4.649533301607972, 4.007130794919371, 4.030215453797171]
